<a href="https://colab.research.google.com/github/MarceloClaro/Computacao-Quantica-Geomaker/blob/main/COMPARACAO_DE_AUTOMATOS_CELULARES_CLASSICOS_E_QUANTICOS_Performance%2C_robustez_ao_ru%C3%ADdo_e_aplicacoes_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar pacotes necessários do Qiskit e Seaborn.
!pip install qiskit>=1.0  # Instala a versão necessária do Qiskit para realizar operações de computação quântica.
!pip install qiskit-ibm-runtime  # Instala a biblioteca qiskit-ibm-runtime para execução de circuitos no IBM Quantum Experience.
!pip install 'qiskit[visualization]'  # Instala o pacote de visualização do Qiskit para visualizações gráficas.
!pip install qiskit_aer  # Instala o simulador Aer do Qiskit para simulações quânticas.
!pip install seaborn  # Instala a biblioteca Seaborn para visualização de dados, especialmente gráficos estatísticos.

# Importando bibliotecas necessárias
import numpy as np  # Importa a biblioteca NumPy, que oferece suporte para arrays e operações numéricas de alto desempenho.
import matplotlib.pyplot as plt  # Importa a biblioteca Matplotlib, que permite a criação de gráficos e visualizações de dados.
import seaborn as sns  # Importa a biblioteca Seaborn, que é baseada no Matplotlib e oferece visualizações estatísticas aprimoradas.
from qiskit import QuantumCircuit, transpile  # Importa classes do Qiskit para criação de circuitos quânticos e transpile para otimização de circuitos.
from qiskit.visualization import plot_histogram, plot_bloch_multivector, circuit_drawer  # Importa funções do Qiskit para visualização de histogramas, vetores de Bloch e circuitos.
from qiskit_aer import AerSimulator  # Importa o simulador Aer do Qiskit, que simula a execução de circuitos quânticos.
import pandas as pd  # Importa a biblioteca pandas, que fornece estruturas de dados de alto desempenho e ferramentas de análise de dados.
from scipy.stats import entropy  # Importa a função de entropia da biblioteca scipy, usada para cálculos estatísticos de entropia.
import time  # Importa a biblioteca time, que fornece funções para medir e manipular o tempo.

# Função para aplicar a regra clássica
def apply_classical_rule(rule_number, ca):
    """
    Aplica uma regra de autômato celular clássico a um estado.

    Parameters:
        rule_number (int): Número da regra a ser aplicada.
        ca (np.array): Array representando o estado atual do autômato.

    Returns:
        np.array: Novo estado do autômato após aplicação da regra.
    """
    # Converte o número da regra para binário e armazena em um array de inteiros
    rule_bin = np.array([int(x) for x in np.binary_repr(rule_number, width=8)], dtype=np.int8)
    # Cria um novo array para armazenar o próximo estado do autômato celular
    new_ca = np.zeros_like(ca)
    # Aplica a regra binária para determinar o próximo estado de cada célula
    for i in range(1, len(ca) - 1):
        new_ca[i] = rule_bin[7 - (ca[i - 1] * 4 + ca[i] * 2 + ca[i + 1])]
    return new_ca

# Função para simular a regra clássica
def simulate_classical(rule_number, steps=100, size=101):
    """
    Simula um autômato celular clássico por um número de passos.

    Parameters:
        rule_number (int): Número da regra a ser aplicada.
        steps (int): Número de passos para simular.
        size (int): Tamanho do autômato.

    Returns:
        np.array: Histórico dos estados do autômato durante a simulação.
    """
    # Inicializa o estado do autômato com todas as células em 0, exceto a do meio
    state = np.zeros(size, dtype=np.int8)
    state[size // 2] = 1
    # Cria um histórico para armazenar o estado em cada passo
    history = np.zeros((steps, size), dtype=np.int8)
    history[0] = state
    # Aplica a regra clássica para cada passo e atualiza o histórico
    for t in range(1, steps):
        state = apply_classical_rule(rule_number, state)
        history[t] = state
    return history

# Funções para adicionar ruído
def add_noise_to_classical(history, noise_level=0.01):
    """
    Adiciona ruído a uma simulação de autômato celular clássico.

    Parameters:
        history (np.array): Histórico dos estados do autômato.
        noise_level (float): Nível de ruído a ser adicionado.

    Returns:
        np.array: Histórico com ruído adicionado.
    """
    # Cria uma cópia do histórico original para adicionar ruído
    noisy_history = history.copy()
    # Gera ruído aleatório baseado no nível de ruído especificado
    noise = np.random.choice([0, 1], size=history.shape, p=[1-noise_level, noise_level])
    # Aplica o ruído ao histórico usando a operação XOR
    noisy_history = np.bitwise_xor(noisy_history, noise)
    return noisy_history

def add_noise_to_quantum_results(quantum_results, noise_level=0.01):
    """
    Adiciona ruído a uma simulação quântica.

    Parameters:
        quantum_results (list): Resultados da simulação quântica.
        noise_level (float): Nível de ruído a ser adicionado.

    Returns:
        list: Resultados com ruído adicionado.
    """
    # Cria uma lista para armazenar os resultados com ruído
    noisy_results = []
    # Itera sobre cada passo da simulação quântica
    for step in quantum_results:
        noisy_step = []
        # Itera sobre cada bloco de resultados
        for block in step:
            noisy_block = {}
            # Aplica ruído a cada estado baseado na contagem original
            for state, count in block.items():
                noisy_count = int(count * (1 - noise_level) + np.random.poisson(noise_level * count))
                noisy_block[state] = noisy_count
            noisy_step.append(noisy_block)
        noisy_results.append(noisy_step)
    return noisy_results

# Função para aplicar uma regra quântica baseada no estado clássico
def apply_quantum_rule_based_on_classical_state(qc, classical_state):
    """
    Aplica uma regra quântica baseada no estado clássico inicial.

    Parameters:
        qc (QuantumCircuit): Circuito quântico a ser configurado.
        classical_state (np.array): Estado clássico inicial.

    Returns:
        QuantumCircuit: Circuito quântico configurado.
    """
    n = len(classical_state)
    # Aplica a porta X (NOT) aos qubits onde o estado clássico é 1
    for i in range(n):
        if classical_state[i] == 1:
            qc.x(i)
    return qc

# Função para converter DataFrame clássico em operações quânticas e executar a simulação
def run_quantum_simulation_from_classical(df_classical, block_size, rule_number):
    """
    Executa simulações quânticas baseadas nos estados clássicos.

    Parameters:
        df_classical (pd.DataFrame): DataFrame com estados clássicos.
        block_size (int): Tamanho do bloco de células.
        rule_number (int): Número da regra clássica aplicada.

    Returns:
        list: Resultados da simulação quântica.
    """
    simulator = AerSimulator()  # Cria um simulador quântico
    results = []  # Lista para armazenar os resultados
    num_blocks = (len(df_classical.columns) + block_size - 1) // block_size  # Calcula o número de blocos necessários
    circuit_count = 1  # Contador para numerar os circuitos

    # Itera sobre cada linha do DataFrame clássico
    for index, row in df_classical.iterrows():
        block_results = []  # Lista para armazenar os resultados de cada bloco
        # Itera sobre cada bloco de células
        for block in range(num_blocks):
            start = block * block_size  # Início do bloco
            end = min(start + block_size, len(df_classical.columns))  # Fim do bloco
            block_state = row.values[start:end]  # Estado do bloco
            qc = QuantumCircuit(end - start, end - start)  # Cria um circuito quântico para o bloco
            qc = apply_quantum_rule_based_on_classical_state(qc, block_state)  # Aplica a regra quântica baseada no estado clássico
            qc.measure(range(end - start), range(end - start))  # Adiciona medições aos qubits

            # Executa a simulação no backend AerSimulator
            aer_sim = AerSimulator()  # Cria um simulador quântico.
            transpiled_qc = transpile(qc, aer_sim)  # Transpila o circuito para o simulador Aer.
            result = aer_sim.run(transpiled_qc, shots=1024, memory=True).result()  # Executa a simulação e obtém os resultados.
            counts = result.get_counts()  # Obtém as contagens dos resultados
            block_results.append(counts)  # Adiciona os resultados do bloco à lista

            # Desenho do circuito com título e subtítulo
            title = f'Circuito Quântico {rule_number} - Bloco {block + 1} - Circuito {circuit_count}'  # Título do circuito
            subtitle = ('Este circuito quântico simula a regra {0}. '
                        'Os qubits são configurados com base no estado inicial clássico e, em seguida, '
                        'medições são realizadas para obter os resultados. Neste circuito, as operações quânticas '
                        'aplicadas incluem portas X (NOT) e medições. As portas X são aplicadas aos qubits quando o '
                        'estado clássico correspondente é 1. Após as operações, os qubits são medidos para determinar '
                        'o estado final do sistema.').format(rule_number)  # Subtítulo explicando o circuito
            fig, ax = plt.subplots(figsize=(12, 6))  # Cria uma figura para o circuito
            ax.set_title(title)  # Define o título da figura
            ax.text(0.5, -0.1, subtitle, ha='center', va='center', transform=ax.transAxes, wrap=True)  # Adiciona o subtítulo à figura
            qc.draw('mpl', ax=ax)  # Desenha o circuito quântico
            plt.show()  # Exibe a figura

            # Visualização da esfera de Bloch
            statevector = result.data(0).get('statevector')  # Obtém o vetor de estado
            if statevector:  # Se o vetor de estado estiver disponível
                plot_bloch_multivector(statevector)  # Plota a esfera de Bloch
                plt.title(f'Esfera de Bloch - Circuito {circuit_count}')  # Define o título da esfera de Bloch
                plt.show()  # Exibe a figura

            circuit_count += 1  # Incrementa o contador de circuitos
        results.append(block_results)  # Adiciona os resultados do bloco à lista de resultados
    return results

# Função para converter resultados em DataFrame
def quantum_results_to_dataframe(quantum_results):
    """
    Converte resultados de simulação quântica em um DataFrame.

    Parameters:
        quantum_results (list): Resultados da simulação quântica.

    Returns:
        pd.DataFrame: DataFrame contendo os resultados.
    """
    data = []  # Lista para armazenar os dados
    # Itera sobre cada passo da simulação
    for step, block_results in enumerate(quantum_results):
        # Itera sobre cada bloco de resultados
        for block_index, result in enumerate(block_results):
            # Armazena o estado e a contagem em uma lista
            for state, count in result.items():
                data.append({"Step": step, "Block": block_index, "State": state, "Count": count})
    return pd.DataFrame(data)  # Converte a lista de dados em um DataFrame

# Função para juntar blocos e criar DataFrame total
def join_quantum_blocks(df_quantum, block_size, total_size):
    """
    Junta blocos de resultados quânticos em um DataFrame total.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame com resultados quânticos.
        block_size (int): Tamanho do bloco de células.
        total_size (int): Tamanho total do autômato.

    Returns:
        pd.DataFrame: DataFrame combinado dos blocos.
    """
    steps = df_quantum['Step'].max() + 1  # Obtém o número de passos na simulação
    combined_data = []  # Lista para armazenar os dados combinados

    # Itera sobre cada passo da simulação
    for step in range(steps):
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        combined_row = []  # Lista para armazenar os dados combinados do passo atual
        # Itera sobre cada bloco de resultados
        for block in range(step_data['Block'].max() + 1):
            block_data = step_data[step_data['Block'] == block]  # Obtém os dados do bloco atual
            if not block_data.empty:  # Se o bloco não estiver vazio
                # Considera o estado mais frequente para o bloco
                state_counts = block_data.groupby('State')['Count'].sum()
                most_frequent_state = state_counts.idxmax()  # Obtém o estado mais frequente
                combined_row.extend([int(bit) for bit in most_frequent_state])  # Adiciona o estado mais frequente à lista combinada
            else:
                combined_row.extend([0] * block_size)  # Adiciona zeros se o bloco estiver vazio
        combined_data.append([step] + combined_row[:total_size])  # Adiciona os dados combinados à lista

    columns = ['Step'] + [f'Qubit_{i}' for i in range(total_size)]  # Define os nomes das colunas
    return pd.DataFrame(combined_data, columns=columns)  # Converte a lista de dados combinados em um DataFrame

# Simulações clássicas
start_time = time.time()  # Inicia a medição do tempo de execução
history_30_classical = simulate_classical(30)  # Executa a simulação clássica da regra 30
classical_30_time = time.time() - start_time  # Calcula o tempo de execução

start_time = time.time()  # Inicia a medição do tempo de execução
history_60_classical = simulate_classical(60)  # Executa a simulação clássica da regra 60
classical_60_time = time.time() - start_time  # Calcula o tempo de execução

start_time = time.time()  # Inicia a medição do tempo de execução
history_90_classical = simulate_classical(90)  # Executa a simulação clássica da regra 90
classical_90_time = time.time() - start_time  # Calcula o tempo de execução

# Armazenar os resultados em DataFrames
df_30_classical = pd.DataFrame(history_30_classical)  # Converte os resultados da regra 30 em um DataFrame
df_60_classical = pd.DataFrame(history_60_classical)  # Converte os resultados da regra 60 em um DataFrame
df_90_classical = pd.DataFrame(history_90_classical)  # Converte os resultados da regra 90 em um DataFrame

# Executar simulações quânticas baseadas nos estados clássicos
block_size = 7  # Define o tamanho do bloco para simulação quântica
total_size = len(df_30_classical.columns)  # Define o tamanho total com base nos dados clássicos

start_time = time.time()  # Inicia a medição do tempo de execução
counts_30_quantum = run_quantum_simulation_from_classical(df_30_classical, block_size, 30)  # Executa a simulação quântica da regra 30
quantum_30_time = time.time() - start_time  # Calcula o tempo de execução

start_time = time.time()  # Inicia a medição do tempo de execução
counts_60_quantum = run_quantum_simulation_from_classical(df_60_classical, block_size, 60)  # Executa a simulação quântica da regra 60
quantum_60_time = time.time() - start_time  # Calcula o tempo de execução

start_time = time.time()  # Inicia a medição do tempo de execução
counts_90_quantum = run_quantum_simulation_from_classical(df_90_classical, block_size, 90)  # Executa a simulação quântica da regra 90
quantum_90_time = time.time() - start_time  # Calcula o tempo de execução

# Converter os resultados das simulações quânticas em DataFrames
df_30_quantum = quantum_results_to_dataframe(counts_30_quantum)  # Converte os resultados da simulação quântica da regra 30 em um DataFrame
df_60_quantum = quantum_results_to_dataframe(counts_60_quantum)  # Converte os resultados da simulação quântica da regra 60 em um DataFrame
df_90_quantum = quantum_results_to_dataframe(counts_90_quantum)  # Converte os resultados da simulação quântica da regra 90 em um DataFrame

# Juntar blocos quânticos e criar DataFrames totais
df_30_quantum_combined = join_quantum_blocks(df_30_quantum, block_size, total_size)  # Junta os blocos da simulação quântica da regra 30
df_60_quantum_combined = join_quantum_blocks(df_60_quantum, block_size, total_size)  # Junta os blocos da simulação quântica da regra 60
df_90_quantum_combined = join_quantum_blocks(df_90_quantum, block_size, total_size)  # Junta os blocos da simulação quântica da regra 90

# Salvar os DataFrames em arquivos CSV
df_30_classical.to_csv('/content/df_30_classical.csv', index=False)  # Salva o DataFrame da simulação clássica da regra 30 em um arquivo CSV
df_60_classical.to_csv('/content/df_60_classical.csv', index=False)  # Salva o DataFrame da simulação clássica da regra 60 em um arquivo CSV
df_90_classical.to_csv('/content/df_90_classical.csv', index=False)  # Salva o DataFrame da simulação clássica da regra 90 em um arquivo CSV
df_30_quantum_combined.to_csv('/content/df_30_quantum_combined.csv', index=False)  # Salva o DataFrame combinado da simulação quântica da regra 30 em um arquivo CSV
df_60_quantum_combined.to_csv('/content/df_60_quantum_combined.csv', index=False)  # Salva o DataFrame combinado da simulação quântica da regra 60 em um arquivo CSV
df_90_quantum_combined.to_csv('/content/df_90_quantum_combined.csv', index=False)  # Salva o DataFrame combinado da simulação quântica da regra 90 em um arquivo CSV

# Função para plotar simulações clássicas
def plot_classical_simulation(history, title):
    """
    Plota a simulação clássica usando Seaborn.

    Parameters:
        history (np.array): Histórico dos estados do autômato.
        title (str): Título do gráfico.
    """
    plt.figure(figsize=(10, 10))  # Cria uma figura para o gráfico
    sns.heatmap(history, cmap='binary', cbar=True)  # Cria um mapa de calor para a simulação clássica
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Posição')  # Define o rótulo do eixo x
    plt.ylabel('Passo')  # Define o rótulo do eixo y
    plt.show()  # Exibe o gráfico

# Função para plotar simulações quânticas
def plot_quantum_simulation(df, title):
    """
    Plota a simulação quântica usando Seaborn.

    Parameters:
        df (pd.DataFrame): DataFrame contendo os resultados da simulação quântica.
        title (str): Título do gráfico.
    """
    steps = df['Step'].unique()  # Obtém os passos únicos da simulação
    df_heatmap = df.drop(columns='Step').set_index(steps)  # Prepara os dados para o mapa de calor
    plt.figure(figsize=(10, 10))  # Cria uma figura para o gráfico
    sns.heatmap(df_heatmap, cmap='viridis', cbar=True)  # Cria um mapa de calor para a simulação quântica
    plt.title(title)  # Define o título do gráfico
    plt.xlabel('Qubit')  # Define o rótulo do eixo x
    plt.ylabel('Passo')  # Define o rótulo do eixo y
    plt.show()  # Exibe o gráfico

# Plotagens das regras clássicas
plot_classical_simulation(df_30_classical.values, 'Simulação Clássica da Regra 30')  # Plota a simulação clássica da regra 30
plot_classical_simulation(df_60_classical.values, 'Simulação Clássica da Regra 60')  # Plota a simulação clássica da regra 60
plot_classical_simulation(df_90_classical.values, 'Simulação Clássica da Regra 90')  # Plota a simulação clássica da regra 90

# Plotagens das regras quânticas
plot_quantum_simulation(df_30_quantum_combined, 'Simulação Quântica da Regra 30')  # Plota a simulação quântica da regra 30
plot_quantum_simulation(df_60_quantum_combined, 'Simulação Quântica da Regra 60')  # Plota a simulação quântica da regra 60
plot_quantum_simulation(df_90_quantum_combined, 'Simulação Quântica da Regra 90')  # Plota a simulação quântica da regra 90

# Função para calcular a entropia dos resultados quânticos
def calculate_entropy(df_quantum):
    """
    Calcula a entropia dos resultados quânticos.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame contendo os resultados quânticos.

    Returns:
        float: Entropia média.
    """
    entropies = []  # Lista para armazenar as entropias
    # Itera sobre cada passo da simulação
    for step in df_quantum['Step'].unique():
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        counts = step_data.drop(columns=['Step']).values.flatten()  # Obtém as contagens dos estados
        entropies.append(entropy(counts[counts > 0]))  # Calcula a entropia e adiciona à lista
    return np.mean(entropies)  # Retorna a entropia média

# Calcula a entropia para cada regra quântica
entropy_30_quantum = calculate_entropy(df_30_quantum_combined)  # Calcula a entropia da simulação quântica da regra 30
entropy_60_quantum = calculate_entropy(df_60_quantum_combined)  # Calcula a entropia da simulação quântica da regra 60
entropy_90_quantum = calculate_entropy(df_90_quantum_combined)  # Calcula a entropia da simulação quântica da regra 90

# Imprime as entropias calculadas
print(f"Entropia da Regra Quântica 30: {entropy_30_quantum}")  # Imprime a entropia da regra 30
print(f"Entropia da Regra Quântica 60: {entropy_60_quantum}")  # Imprime a entropia da regra 60
print(f"Entropia da Regra Quântica 90: {entropy_90_quantum}")  # Imprime a entropia da regra 90

# Robustez ao Ruído
# Adiciona ruído aos resultados quânticos e recalcula a entropia
noisy_counts_30_quantum = add_noise_to_quantum_results(counts_30_quantum)  # Adiciona ruído à simulação quântica da regra 30
noisy_counts_60_quantum = add_noise_to_quantum_results(counts_60_quantum)  # Adiciona ruído à simulação quântica da regra 60
noisy_counts_90_quantum = add_noise_to_quantum_results(counts_90_quantum)  # Adiciona ruído à simulação quântica da regra 90

df_noisy_30_quantum = quantum_results_to_dataframe(noisy_counts_30_quantum)  # Converte os resultados ruidosos da regra 30 em um DataFrame
df_noisy_60_quantum = quantum_results_to_dataframe(noisy_counts_60_quantum)  # Converte os resultados ruidosos da regra 60 em um DataFrame
df_noisy_90_quantum = quantum_results_to_dataframe(noisy_counts_90_quantum)  # Converte os resultados ruidosos da regra 90 em um DataFrame

df_noisy_30_quantum_combined = join_quantum_blocks(df_noisy_30_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 30
df_noisy_60_quantum_combined = join_quantum_blocks(df_noisy_60_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 60
df_noisy_90_quantum_combined = join_quantum_blocks(df_noisy_90_quantum, block_size, total_size)  # Junta os blocos ruidosos da regra 90

noisy_entropy_30_quantum = calculate_entropy(df_noisy_30_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 30
noisy_entropy_60_quantum = calculate_entropy(df_noisy_60_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 60
noisy_entropy_90_quantum = calculate_entropy(df_noisy_90_quantum_combined)  # Calcula a entropia dos resultados ruidosos da regra 90

# Imprime as entropias calculadas com ruído
print(f"Entropia da Regra Quântica 30 com Ruído: {noisy_entropy_30_quantum}")  # Imprime a entropia ruidosa da regra 30
print(f"Entropia da Regra Quântica 60 com Ruído: {noisy_entropy_60_quantum}")  # Imprime a entropia ruidosa da regra 60
print(f"Entropia da Regra Quântica 90 com Ruído: {noisy_entropy_90_quantum}")  # Imprime a entropia ruidosa da regra 90

# Plotar entropia em cada estado das regras quânticas
def plot_entropy_per_step(df_quantum, title):
    """
    Plota a entropia para cada passo da simulação quântica.

    Parameters:
        df_quantum (pd.DataFrame): DataFrame contendo os resultados quânticos.
        title (str): Título do gráfico.
    """
    entropies = []  # Lista para armazenar as entropias
    steps = df_quantum['Step'].unique()  # Obtém os passos únicos da simulação
    # Itera sobre cada passo da simulação
    for step in steps:
        step_data = df_quantum[df_quantum['Step'] == step]  # Obtém os dados do passo atual
        counts = step_data.drop(columns=['Step']).values.flatten()  # Obtém as contagens dos estados
        entropies.append(entropy(counts[counts > 0]))  # Calcula a entropia e adiciona à lista
    # Plota a entropia para cada passo da simulação
    plt.figure(figsize=(10, 6))
    plt.plot(steps, entropies, marker='o')
    plt.title(title)
    plt.xlabel('Passo')
    plt.ylabel('Entropia')
    plt.grid(True)
    plt.show()

# Plota a entropia para cada passo das regras quânticas
plot_entropy_per_step(df_30_quantum_combined, 'Entropia por Passo - Regra Quântica 30')  # Plota a entropia da regra 30
plot_entropy_per_step(df_60_quantum_combined, 'Entropia por Passo - Regra Quântica 60')  # Plota a entropia da regra 60
plot_entropy_per_step(df_90_quantum_combined, 'Entropia por Passo - Regra Quântica 90')  # Plota a entropia da regra 90

# Medições de tempo de execução para as simulações
print(f"Tempo de execução para Regra Clássica 30: {classical_30_time} segundos")  # Imprime o tempo de execução da regra clássica 30
print(f"Tempo de execução para Regra Clássica 60: {classical_60_time} segundos")  # Imprime o tempo de execução da regra clássica 60
print(f"Tempo de execução para Regra Clássica 90: {classical_90_time} segundos")  # Imprime o tempo de execução da regra clássica 90

print(f"Tempo de execução para Regra Quântica 30: {quantum_30_time} segundos")  # Imprime o tempo de execução da regra quântica 30
print(f"Tempo de execução para Regra Quântica 60: {quantum_60_time} segundos")  # Imprime o tempo de execução da regra quântica 60
print(f"Tempo de execução para Regra Quântica 90: {quantum_90_time} segundos")  # Imprime o tempo de execução da regra quântica 90

# Carregar os dados dos arquivos CSV
df_30_classical_loaded = pd.read_csv('/content/df_30_classical.csv')  # Carrega os dados da simulação clássica da regra 30
df_60_classical_loaded = pd.read_csv('/content/df_60_classical.csv')  # Carrega os dados da simulação clássica da regra 60
df_90_classical_loaded = pd.read_csv('/content/df_90_classical.csv')  # Carrega os dados da simulação clássica da regra 90
df_30_quantum_combined_loaded = pd.read_csv('/content/df_30_quantum_combined.csv')  # Carrega os dados da simulação quântica da regra 30
df_60_quantum_combined_loaded = pd.read_csv('/content/df_60_quantum_combined.csv')  # Carrega os dados da simulação quântica da regra 60
df_90_quantum_combined_loaded = pd.read_csv('/content/df_90_quantum_combined.csv')  # Carrega os dados da simulação quântica da regra 90

# Função para exibir os dados carregados dos arquivos CSV
def display_csv_data(df, title):
    """
    Exibe os dados carregados dos arquivos CSV como uma tabela.

    Parameters:
        df (pd.DataFrame): DataFrame contendo os dados carregados.
        title (str): Título da tabela.
    """
    print(f"\n{title}")  # Imprime o título da tabela
    display(df)  # Exibe a tabela

# Exibir as tabelas dos dados carregados
display_csv_data(df_30_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 30')  # Exibe a tabela da simulação clássica da regra 30
display_csv_data(df_60_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 60')  # Exibe a tabela da simulação clássica da regra 60
display_csv_data(df_90_classical_loaded, 'Tabela CSV - Simulação Clássica da Regra 90')  # Exibe a tabela da simulação clássica da regra 90

display_csv_data(df_30_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 30')  # Exibe a tabela da simulação quântica da regra 30
display_csv_data(df_60_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 60')  # Exibe a tabela da simulação quântica da regra 60
display_csv_data(df_90_quantum_combined_loaded, 'Tabela CSV - Simulação Quântica da Regra 90')  # Exibe a tabela da simulação quântica da regra 90
